In [0]:
import pyspark
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import *

In [0]:
orders_df = spark.read.format("parquet") \
    .load("abfss://bronze@salesstaradlsstorageacc.dfs.core.windows.net/orders")

In [0]:
orders_df.display()  # Check the DataFrame's source path and configuration for correct scheme.

In [0]:
orders_df =orders_df.withColumnRenamed("_rescued_data","rescued_data")

In [0]:
orders_df.display(10)

In [0]:
orders_df = orders_df.drop("rescued_data")

###Date Transformations


In [0]:
orders_df = orders_df.withColumn("order_date",to_timestamp(col("order_date")))
orders_df.display()

In [0]:
orders_df = orders_df.withColumn("year",year(col("order_date")))



In [0]:
orders_df = orders_df.withColumn("month",month(col("order_date")))

In [0]:
orders_df.display()

In [0]:
df1 = orders_df.withColumn("dense_rank_flag", dense_rank().over(Window.partitionBy("year").orderBy(desc("total_amount"))))

In [0]:
df1.drop

In [0]:
df2 = orders_df.withColumn("rank_flag",rank().over(Window.partitionBy("year").orderBy(desc("total_amount"))))
df2.display()

In [0]:
df3 = orders_df.withColumn("row_number_flag",row_number().over(Window.partitionBy("year").orderBy(desc("total_amount"))))

In [0]:
df3.display()

In [0]:
class windowfunction():

    def __init__(self,df):
        self.df = df
    
    def ranked_df(self):

        return self.df.withColumn("ranked_flag",rank().over(Window.partitionBy(col("year")).orderBy(desc(col("total_amount")))))
    

    def dense_rank_df(self):

        return self.df.withColumn("dense_rank_flag",dense_rank().over(Window.partitionBy(col("year")).orderBy(desc(col("total_amount")))))
    
    def row_number_df(self):
    
        return self.df.withColumn("row_number_flag",row_number().over(Window.partitionBy(col("year")).orderBy(desc(col("total_amount")))))

In [0]:
wf = windowfunction(orders_df)

ranked_df1 = wf.ranked_df()
ranked_df1.display()

dense_ranked_df1 = wf.dense_rank_df()
dense_ranked_df1.display()

row_number_df1 = wf.row_number_df()
row_number_df1.display()

Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.

In [0]:
row_number_df1.display()

In [0]:
row_number_df1.write \
    .mode("append") \
        .format("delta") \
            .save("abfss://silver@salesstaradlsstorageacc.dfs.core.windows.net/orders")

In [0]:
%sql
create table if not exists databricksete.silver.orders_silver
using delta 
location "abfss://silver@salesstaradlsstorageacc.dfs.core.windows.net/orders"